### Basic expansion for different organisms

Loading the necessary packages and setting the working directory.

In [99]:
cd(joinpath(homedir(), "Macdocs", "Master", "Internship_JP", "metnetexp", "BioXP"))

using Pkg
Pkg.activate(joinpath(pwd(), "Project.toml"))

using BioXP, ProgressMeter, Random, JSON, InteractiveUtils


  Activating project at `~/Macdocs/Master/Internship_JP/metnetexp/BioXP`


Input directory structure, including to-be-used files

In [100]:
#Directory/Folder/File name variables
#Names
organismtype = "methanogens"
sidname = "Enceladus_20-SAFR-032"
tidname = "targets_Freilich09"


#Files
rstructsfile = "master_from_redges-og-submission.json"
seedsfile = "seeds.json"
targetsfile = "seeds.json"

#Folders
ridsdir = "rids-methanogens"
seedsdir = "seeds"
targetsdir = "targets"
compoundsdir = "compounds"


#All directory paths. No need to change these unless the structure of the directories change
inputdir = joinpath(pwd(),"data","input")

seedsdir = joinpath(inputdir, seedsdir)
targetsdir = joinpath(inputdir, targetsdir)
ridsdir = joinpath(inputdir, ridsdir)
compoundsdir = joinpath(inputdir, compoundsdir)


outputdir = joinpath(pwd(),"data","output")
writedir = joinpath(inputdir, "sid_randomizations", "sidrand"*"_$organismtype")

#Creates writing directory, if it doesn't exist
!ispath(writedir) && mkpath(writedir);

#All file paths
rstructspath = joinpath(inputdir, rstructsfile)
seedspath = joinpath(seedsdir, seedsfile)
targetspath = joinpath(seedsdir, targetsfile);

Function to get and print directory and file paths

In [101]:
"""
This function prints the directory and file paths 
that are used in the script, to check if they are correct
Gets all the arguments in the Main module, 
filters for the ones that end with "dir" or "path"
and prints them in a readable format
"""

function getandprintDirPaths()

    vars_dict = Dict()

    for var in names(Main)

        #Checks if the variable is a directory or file path
        if endswith(string(var), "dir") || endswith(string(var), "path")
            
    
            if endswith(string(var), "dir") 
                tbprinted = join(split(eval(var), "/")[end-2:end], "/") 
                dict_key_end = " Directory: "
                dict_key_beg = uppercasefirst(split(string(var), "dir")[1])
            else
                tbprinted = join(split(eval(var), "/")[end:end], "/")
                dict_key_end = " file path: "
                dict_key_beg = uppercasefirst(split(string(var), "path")[1])
            end

            vars_dict[dict_key_beg*dict_key_end] = tbprinted
            
        end
    end

    for key in keys(vars_dict)
        println(key, vars_dict[key], "\n")
    end
end



getandprintDirPaths (generic function with 1 method)

Variable determination and 
seed set randomization

In [126]:
#Get all the seed filepaths and filenames
seed_filepaths = filter(x -> !startswith(x, "."), readdir(ridsdir, join=true))
seed_filenames = filter(x -> !startswith(x, "."), readdir(ridsdir));

#Create usable data structures from files based on parser functions
sids = readkeyedids(seedspath)[sidname]
tids = readkeyedids(targetspath)[tidname]

rstructs = readmaster(rstructspath)
compoundstructs = readcompounds(compoundsdir)


#Set dict of arguments for the randomization compounds function
rand_comp_args = Dict(
    :compound_structs => compoundstructs :: Compounds,
    :tids => tids :: IDs,
    :n_runs => 100 :: Int64,
    :n_swaps => 1000 :: Int64,
    :beta => 20 :: Real,
    :sortkey => :exact_mass :: Symbol,
    :rng => MersenneTwister(1234)
);

#(;args...)

LoadError: TypeError: in typeassert, expected AbstractRNG, got a value of type Int64

In [125]:
#Running the actual generation of random seed sets
Threads.@threads for (i, file_name) in collect(enumerate(seed_filenames))

    p = Progress(length(file_name), desc=file_name)

    cids = BioXP.identify_biosystem_compounds(rstructs, BioXP.remove_rids_not_in_rstructs(rstructs, readids(seed_filepaths[i])))



    rand_seed_sets = randomizecompounds(cids; rand_comp_args...)
    
    # open(joinpath(writedir, seed_filenames[i]), "w") do f
    #     JSON.print(f, random_seed_sets, 2)
    # end

    # next!(p)

end

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching randomizecompounds(::Vector{String}; beta::Int64, n_swaps::Int64, n_runs::Int64, rng::Int64, tids::Vector{String}, sortkey::Symbol, compound_structs::Dict{String, Compound})
    
    [0mClosest candidates are:
    [0m  randomizecompounds(::Vector{String}, [91m::Vector{String}[39m, [91m::Dict{String, Compound}[39m, [91m::Int64[39m, [91m::Int64[39m)[91m got unsupported keyword arguments "beta", "n_swaps", "n_runs", "rng", "tids", "sortkey", "compound_structs"[39m
    [0m[90m   @[39m [35mBioXP[39m [90m~/Macdocs/Master/Internship_JP/metnetexp/BioXP/src/[39m[90m[4mrandomize.jl:128[24m[39m
    [0m  randomizecompounds(::Vector{String}, [91m::Vector{String}[39m, [91m::Dict{String, Compound}[39m, [91m::Int64[39m)[91m got unsupported keyword arguments "beta", "n_swaps", "n_runs", "rng", "tids", "sortkey", "compound_structs"[39m
    [0m[90m   @[39m [35mBioXP[39m [

In [51]:
# for i in 1:Threads.nthreads()
#     println("i: ", i, "\t Thread ID: ", Threads.threadid()) 
# end

i: 1	 Thread ID: 1
